In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import matplotlib.pyplot as plt
import glob

/home/asphyxia/anaconda3/envs/tensorflow/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
a = "train/input_node_files/sce3a"
a[-5::]

'sce3a'

In [4]:
results = pd.DataFrame()
input_path = "train/input_node_files/"
output_path = "train/output_simulator/"


directory = input_path  # Replace with the actual directory path

# Get the files in the directory
files = os.listdir(directory)
files.remove(".DS_Store")
files.sort()

# Iterate over the files and get their full paths
file_paths = [os.path.join(directory, file) for file in files]

dfs = []
combined_df = pd.DataFrame()
# Print the file paths
for file_path in file_paths:
    for i in range(80):
        input_file = file_path+"/"+"input_nodes_"+file_path[-5::]+"_deployment_"+str(i).zfill(3)+".csv"
        df = pd.read_csv(input_file)
        dfs.append(df)   
combined_df = pd.concat(dfs,ignore_index=True)         
 
combined_df.to_csv('Combined.csv', index=False)

In [5]:
# results.head()

In [6]:
input_df = pd.read_csv("Combined.csv")
input_df.head()

,node_code;node_type;wlan_code;x(m);y(m);z(m);central_freq(GHz);channel_bonding_model;primary_channel;min_channel_allowed;max_channel_allowed;tpc_default(dBm);cca_default(dBm);traffic_model;traffic_load[pkt/s];packet_length;num_packets_aggregated;capture_effect_model;capture_effect_thr;constant_per;pifs_activated;cw_adaptation;cont_wind;cont_wind_stage
0,AP_A;0;A;7.5;8.3333;8.3417;5.0;4;0;0;3;20;-82;...
1,STA_A1;1;A;12.0627;4.6918;5.067;5.0;4;0;0;3;20...
2,STA_A2;1;A;8.2712;4.8383;9.9676;5.0;4;0;0;3;20...
3,STA_A3;1;A;8.4824;3.5287;8.9153;5.0;4;0;0;3;20...
4,STA_A4;1;A;1.3639;1.825;8.3635;5.0;4;0;0;3;20;...


In [7]:
input_df = pd.read_csv("Combined.csv", delimiter=";")
input_df.head()

,node_code,node_type,wlan_code,x(m),y(m),z(m),central_freq(GHz),channel_bonding_model,primary_channel,min_channel_allowed,...,traffic_load[pkt/s],packet_length,num_packets_aggregated,capture_effect_model,capture_effect_thr,constant_per,pifs_activated,cw_adaptation,cont_wind,cont_wind_stage
0,AP_A,0,A,7.5000,8.3333,8.3417,5.0,4,0,0,...,10000.0,12000,64,0,10,0.0,0,0,16,5
1,STA_A1,1,A,12.0627,4.6918,5.0670,5.0,4,0,0,...,10000.0,12000,64,0,10,0.0,0,0,16,5
2,STA_A2,1,A,8.2712,4.8383,9.9676,5.0,4,0,0,...,10000.0,12000,64,0,10,0.0,0,0,16,5
3,STA_A3,1,A,8.4824,3.5287,8.9153,5.0,4,0,0,...,10000.0,12000,64,0,10,0.0,0,0,16,5
4,STA_A4,1,A,1.3639,1.8250,8.3635,5.0,4,0,0,...,10000.0,12000,64,0,10,0.0,0,0,16,5


In [8]:
input_df.head(50)

,node_code,node_type,wlan_code,x(m),y(m),z(m),central_freq(GHz),channel_bonding_model,primary_channel,min_channel_allowed,...,traffic_load[pkt/s],packet_length,num_packets_aggregated,capture_effect_model,capture_effect_thr,constant_per,pifs_activated,cw_adaptation,cont_wind,cont_wind_stage
0,AP_A,0,A,7.5000,8.3333,8.3417,5.0,4,0,0,...,10000.0,12000,64,0,10,0.0,0,0,16,5
1,STA_A1,1,A,12.0627,4.6918,5.0670,5.0,4,0,0,...,10000.0,12000,64,0,10,0.0,0,0,16,5
2,STA_A2,1,A,8.2712,4.8383,9.9676,5.0,4,0,0,...,10000.0,12000,64,0,10,0.0,0,0,16,5
3,STA_A3,1,A,8.4824,3.5287,8.9153,5.0,4,0,0,...,10000.0,12000,64,0,10,0.0,0,0,16,5
4,STA_A4,1,A,1.3639,1.8250,8.3635,5.0,4,0,0,...,10000.0,12000,64,0,10,0.0,0,0,16,5
5,STA_A5,1,A,1.3786,3.2654,8.2911,5.0,4,0,0,...,10000.0,12000,64,0,10,0.0,0,0,16,5
6,STA_A6,1,A,1.0444,1.2486,8.0232,5.0,4,0,0,...,10000.0,12000,64,0,10,0.0,0,0,16,5
7,STA_A7,1,A,0.0853,13.7302,8.3594,5.0,4,0,0,...,10000.0,12000,64,0,10,0.0,0,0,16,5
8,STA_A8,1,A,7.8169,18.2302,5.4278,5.0,4,0,0,...,10000.0,12000,64,0,10,0.0,0,0,16,5
9,STA_A9,1,A,9.9062,5.0489,3.0879,5.0,4,0,0,...,10000.0,12000,64,0,10,0.0,0,0,16,5


In [9]:
output_path = "train/output_simulator/"
output_dirs = os.listdir(output_path)
output_dirs.remove(".DS_Store")
output_dirs.sort()

In [10]:
output_dirs

['sce3a_output',
 'sce3b_output',
 'sce3c_output',
 'sce3d_output',
 'sce4a_output',
 'sce4b_output',
 'sce4c_output',
 'sce4d_output',
 'sce5a_output',
 'sce5b_output',
 'sce5c_output',
 'sce5d_output',
 'sce6a_output',
 'sce6b_output',
 'sce6c_output',
 'sce6d_output',
 'sce7a_output',
 'sce7b_output',
 'sce7c_output',
 'sce7d_output',
 'sce8a_output',
 'sce8b_output',
 'sce8c_output',
 'sce8d_output']

In [11]:
throughput = []
airtime = []
RSSI = []
interference_map = []
SINR = []

In [12]:
ap12 = ['3a','3b','3c','3d','4a','4b','4c','4d']
ap10 = ['5a','5b','5c','5d','6a','6b','6c','6d']
ap8 = ['7a','7b','7c','7d','8a','8b','8c','8d']

In [13]:
output_dirs.sort()

In [14]:
# #go through ever output folder and open each csv file
# for i in output_dirs:
#     for file in os.listdir(output_path+i):
#         f = open(output_path+i+"/"+file,"r+")
#         print(i)
#         # print(file)
#         f.seek(0)
#         data = f.read().rstrip()
#         txt = "{" + data +"}"
#         f.seek(0)
#         f.write(txt)
#         f.close()


# file = "train/output_simulator/sce3a_output/airtime_1.csv"
# f = open(file, 'r+')
# f.seek(0)
# data = f.read().rstrip()
# txt = "{" + data +"}"
# f.seek(0)
# f.write(txt)
# f.close()


In [15]:
input_dirs = os.listdir("train/input_node_files/")
input_dirs.remove(".DS_Store")

In [16]:
output_dirs

['sce3a_output',
 'sce3b_output',
 'sce3c_output',
 'sce3d_output',
 'sce4a_output',
 'sce4b_output',
 'sce4c_output',
 'sce4d_output',
 'sce5a_output',
 'sce5b_output',
 'sce5c_output',
 'sce5d_output',
 'sce6a_output',
 'sce6b_output',
 'sce6c_output',
 'sce6d_output',
 'sce7a_output',
 'sce7b_output',
 'sce7c_output',
 'sce7d_output',
 'sce8a_output',
 'sce8b_output',
 'sce8c_output',
 'sce8d_output']

In [17]:
input_dirs

['sce3b',
 'sce6d',
 'sce3d',
 'sce4c',
 'sce4a',
 'sce4b',
 'sce8d',
 'sce8b',
 'sce5c',
 'sce7c',
 'sce7a',
 'sce5b',
 'sce5d',
 'sce7d',
 'sce4d',
 'sce3a',
 'sce8a',
 'sce6a',
 'sce5a',
 'sce8c',
 'sce6b',
 'sce3c',
 'sce7b',
 'sce6c']

In [18]:
input_dirs.sort()

In [19]:
import os
import pandas as pd

airtime = []
throughput = []
sinr = []
rssi = []
interference_map = []

# for folder in output_dirs:
#     for file in os.listdir(os.path.join(output_path,folder)):
#         if 'sinr' in file:
#             f = open(os.path.join(output_path,folder,file))
#             f.seek(0)
#             data = f.read()
#             sinr.append(data)
#         if 'throughput' in file:
#             f = open(os.path.join(output_path,folder,file))
#             f.seek(0)
#             data = f.read()
#             throughput.append(data)
#         if 'airtime' in file:
#             f = open(os.path.join(output_path,folder,file))
#             f.seek(0)
#             data = f.read()
#             airtime.append(data)
#         if 'rssi' in file:
#             f = open(os.path.join(output_path,folder,file))
#             f.seek(0)
#             data = f.read()
#             rssi.append(data)
#         if 'interference' in file:
#             f = open(os.path.join(output_path,folder,file))
#             f.seek(0)
#             data = f.read()
#             interference_map.append(data)

for folder in output_dirs:
    for i in range(1,81):
        airtime_path = output_path+folder+"/"+"airtime_"+str(i)+".csv"
        f = open(airtime_path)
        f.seek(0)
        data = f.read()
        data_string = data.strip('{}')

        # Split the string by semicolons to get rows
        rows = data_string.split(';')

        # Process each row and split the string by commas
        data_list = []
        for row in rows:
            # Split the row by commas, skipping empty strings, and convert each non-empty element to float
            float_row = [float(num) for num in row.split(',') if num]
            airtime.append(float_row)
        airtime.pop()
        f.close()

        throughput_path = output_path+folder+"/"+"throughput_"+str(i)+".csv"
        f = open(throughput_path)
        f.seek(0)
        data = f.read()
        data_string = data.strip('{}')

        # Split the string by semicolons to get rows
        rows = data_string.split(';')

        # Process each row and split the string by commas
        data_list = []
        for row in rows:
            # Split the row by commas, skipping empty strings, and convert each non-empty element to float
            float_row = [float(num) for num in row.split(',') if num]
            throughput.append(float_row)
        f.close()

        sinr_path = output_path+folder+"/"+"sinr_"+str(i)+".csv"
        f = open(sinr_path)
        f.seek(0)
        data = f.read()
        data_string = data.strip('{}')

        # Split the string by semicolons to get rows
        rows = data_string.split(';')

        # Process each row and split the string by commas
        data_list = []
        for row in rows:
            # Split the row by commas, skipping empty strings, and convert each non-empty element to float
            float_row = [float(num) for num in row.split(',') if num]
            sinr.append(float_row)
        f.close()
        
        rssi_path = output_path+folder+"/"+"rssi_"+str(i)+".csv"
        f = open(rssi_path)
        f.seek(0)
        data = f.read()
        data_string = data.strip('{}')

        # Split the string by semicolons to get rows
        rows = data_string.split(';')

        # Process each row and split the string by commas
        data_list = []
        for row in rows:
            # Split the row by commas, skipping empty strings, and convert each non-empty element to float
            float_row = [float(num) for num in row.split(',') if num]
            rssi.append(float_row)
        f.close()

        interference_map_path = output_path+folder+"/"+"interference_"+str(i)+".csv"
        f = open(interference_map_path)
        f.seek(0)
        data = f.read()
        data_string = data.strip('{}')

        # Split the string by semicolons to get rows
        rows = data_string.split(';')

        # Process each row and split the string by commas
        data_list = []
        for row in rows:
            # Split the row by commas, skipping empty strings, and convert each non-empty element to float
            float_row = [float(num) for num in row.split(',') if num]
            interference_map.append(float_row)
        f.close()

In [20]:
sum = 0
for element in throughput:
    sum += len(element)
sum

306424

In [27]:
airtime[12]

[89.44, 74.63, 45.07, 45.07]

In [28]:
input_df.shape[0]

306424

In [29]:
throughput[-1]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [30]:
added_output_df = input_df.copy()
added_output_df.head()

,node_code,node_type,wlan_code,x(m),y(m),z(m),central_freq(GHz),channel_bonding_model,primary_channel,min_channel_allowed,...,traffic_load[pkt/s],packet_length,num_packets_aggregated,capture_effect_model,capture_effect_thr,constant_per,pifs_activated,cw_adaptation,cont_wind,cont_wind_stage
0,AP_A,0,A,7.5000,8.3333,8.3417,5.0,4,0,0,...,10000.0,12000,64,0,10,0.0,0,0,16,5
1,STA_A1,1,A,12.0627,4.6918,5.0670,5.0,4,0,0,...,10000.0,12000,64,0,10,0.0,0,0,16,5
2,STA_A2,1,A,8.2712,4.8383,9.9676,5.0,4,0,0,...,10000.0,12000,64,0,10,0.0,0,0,16,5
3,STA_A3,1,A,8.4824,3.5287,8.9153,5.0,4,0,0,...,10000.0,12000,64,0,10,0.0,0,0,16,5
4,STA_A4,1,A,1.3639,1.8250,8.3635,5.0,4,0,0,...,10000.0,12000,64,0,10,0.0,0,0,16,5


In [31]:
# print(throughput[55])

In [32]:
print(throughput[960])

[18.77, 5.12, 0.85, 0.0, 2.56, 0.85, 0.0, 0.0, 0.0, 0.85, 0.0, 0.0, 0.85, 1.71, 0.0, 5.12, 0.0, 0.85, 58.03, 0.0, 2.56, 0.85, 5.12, 5.12, 0.0, 1.71, 2.56, 1.71, 10.24, 0.0, 6.83, 4.27, 4.27, 2.56, 3.41, 0.85, 2.56, 3.41, 112.13, 11.95, 8.36, 18.77, 8.0, 7.6, 11.4, 15.36, 19.63, 2.53, 8.53, 42.67, 0.0, 0.85, 7.68, 0.0, 5.12, 0.85, 1.71, 8.53, 1.71, 0.85, 2.56, 5.12, 1.71, 1.71, 0.85, 0.85, 1.71, 0.85, 0.0, 79.77, 2.53, 4.0, 7.68, 5.32, 7.33, 4.56, 3.8, 2.03, 3.04, 9.39, 7.68, 4.56, 5.32, 3.04, 3.41, 6.08, 21.71, 1.33, 0.51, 2.28, 0.51, 0.0, 2.56, 0.0, 1.71, 0.76, 2.37, 0.0, 0.0, 0.0, 0.85, 2.56, 2.37, 3.89, 21.29, 0.0, 0.51, 0.67, 3.41, 2.56, 1.71, 1.52, 1.52, 2.28, 0.0, 3.04, 2.56, 1.52, 102.52, 5.69, 7.59, 5.79, 7.31, 12.24, 5.12, 6.83, 4.27, 8.53, 6.83, 4.27, 14.51, 5.12, 6.73, 1.71, 101.17, 7.97, 11.09, 3.8, 7.12, 11.09, 9.39, 5.97, 8.53, 5.95, 7.68, 5.32, 3.04, 5.95, 4.0, 4.27, 72.24, 2.03, 6.84, 3.8, 4.67, 0.67, 1.52, 3.8, 0.85, 4.56, 2.03, 4.0, 3.04, 4.56, 7.68, 6.83, 3.41, 5.12,

In [33]:
added_output_df.tail(20)

,node_code,node_type,wlan_code,x(m),y(m),z(m),central_freq(GHz),channel_bonding_model,primary_channel,min_channel_allowed,...,traffic_load[pkt/s],packet_length,num_packets_aggregated,capture_effect_model,capture_effect_thr,constant_per,pifs_activated,cw_adaptation,cont_wind,cont_wind_stage
306404,STA_G8,1,G,22.7216,21.3142,4.9312,2.4,6,0,0,...,10000.0,12000,64,0,10,0.0,0,0,16,5
306405,STA_G9,1,G,25.5277,17.9986,8.9464,2.4,6,0,0,...,10000.0,12000,64,0,10,0.0,0,0,16,5
306406,STA_G10,1,G,32.4105,16.6114,5.6497,2.4,6,0,0,...,10000.0,12000,64,0,10,0.0,0,0,16,5
306407,STA_G11,1,G,18.9305,15.1391,6.3869,2.4,6,0,0,...,10000.0,12000,64,0,10,0.0,0,0,16,5
306408,STA_G12,1,G,24.4435,5.8857,5.3953,2.4,6,0,0,...,10000.0,12000,64,0,10,0.0,0,0,16,5
306409,STA_G13,1,G,29.1278,7.8165,6.5244,2.4,6,0,0,...,10000.0,12000,64,0,10,0.0,0,0,16,5
306410,STA_G14,1,G,20.1323,22.7992,9.6976,2.4,6,0,0,...,10000.0,12000,64,0,10,0.0,0,0,16,5
306411,STA_G15,1,G,25.3474,6.0606,9.4454,2.4,6,0,0,...,10000.0,12000,64,0,10,0.0,0,0,16,5
306412,AP_H,0,H,35.0000,15.0000,7.4200,2.4,6,4,4,...,10000.0,12000,64,0,10,0.0,0,0,16,5
306413,STA_H1,1,H,30.0479,12.9123,3.1779,2.4,6,4,4,...,10000.0,12000,64,0,10,0.0,0,0,16,5


In [34]:
len(throughput)

1920

In [35]:
added_output_df.tail(100)

,node_code,node_type,wlan_code,x(m),y(m),z(m),central_freq(GHz),channel_bonding_model,primary_channel,min_channel_allowed,...,traffic_load[pkt/s],packet_length,num_packets_aggregated,capture_effect_model,capture_effect_thr,constant_per,pifs_activated,cw_adaptation,cont_wind,cont_wind_stage
306324,STA_B9,1,B,17.0637,9.4243,3.6383,2.4,6,6,6,...,10000.0,12000,64,0,10,0.0,0,0,16,5
306325,STA_B10,1,B,11.3165,1.9164,5.5010,2.4,6,6,6,...,10000.0,12000,64,0,10,0.0,0,0,16,5
306326,STA_B11,1,B,20.5482,5.7904,4.9398,2.4,6,6,6,...,10000.0,12000,64,0,10,0.0,0,0,16,5
306327,STA_B12,1,B,19.9816,9.4062,6.3751,2.4,6,6,6,...,10000.0,12000,64,0,10,0.0,0,0,16,5
306328,STA_B13,1,B,22.4706,9.2152,5.9611,2.4,6,6,6,...,10000.0,12000,64,0,10,0.0,0,0,16,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306419,STA_H7,1,H,43.8570,13.9867,7.1174,2.4,6,4,4,...,10000.0,12000,64,0,10,0.0,0,0,16,5
306420,STA_H8,1,H,33.9621,21.7909,5.6933,2.4,6,4,4,...,10000.0,12000,64,0,10,0.0,0,0,16,5
306421,STA_H9,1,H,42.1158,13.5457,7.3812,2.4,6,4,4,...,10000.0,12000,64,0,10,0.0,0,0,16,5
306422,STA_H10,1,H,40.0639,8.4542,4.6942,2.4,6,4,4,...,10000.0,12000,64,0,10,0.0,0,0,16,5


In [36]:
len(throughput[0])

170

In [37]:
throughput[:2]

[[104.96,
  7.68,
  11.09,
  14.51,
  11.95,
  10.24,
  11.09,
  9.39,
  3.41,
  13.65,
  11.95,
  52.05,
  5.97,
  0.85,
  5.12,
  0.85,
  5.97,
  5.12,
  7.68,
  5.12,
  0.0,
  7.68,
  0.0,
  0.85,
  3.41,
  1.71,
  1.71,
  40.11,
  0.0,
  3.41,
  1.71,
  1.71,
  7.68,
  1.71,
  2.56,
  1.71,
  2.56,
  0.0,
  1.71,
  3.41,
  1.71,
  3.41,
  3.41,
  3.41,
  125.44,
  10.24,
  16.21,
  11.09,
  14.51,
  5.12,
  2.56,
  7.68,
  10.24,
  12.8,
  7.68,
  15.36,
  11.95,
  34.13,
  0.0,
  5.97,
  0.85,
  3.41,
  0.85,
  2.56,
  2.56,
  8.53,
  4.27,
  0.85,
  0.0,
  4.27,
  18.77,
  0.0,
  2.56,
  1.71,
  2.56,
  2.56,
  0.0,
  4.27,
  0.85,
  1.71,
  0.0,
  2.56,
  44.37,
  2.56,
  2.56,
  2.56,
  3.41,
  5.97,
  0.0,
  1.71,
  11.09,
  0.0,
  0.0,
  10.24,
  0.85,
  0.85,
  2.56,
  27.17,
  1.33,
  0.0,
  0.67,
  0.67,
  0.51,
  3.04,
  0.76,
  0.0,
  0.0,
  2.56,
  0.0,
  4.27,
  5.12,
  2.28,
  5.97,
  90.45,
  5.97,
  9.39,
  5.97,
  8.53,
  11.95,
  2.56,
  6.83,
  8.53,
  5.97,
  2.

In [38]:
added_output_df["throughput"] = np.nan

In [39]:
airtime[0]

[50.3, 50.3, 50.3, 50.3]

In [40]:
added_output_df.shape[0]

306424

In [41]:
added_output_df.tail()

,node_code,node_type,wlan_code,x(m),y(m),z(m),central_freq(GHz),channel_bonding_model,primary_channel,min_channel_allowed,...,packet_length,num_packets_aggregated,capture_effect_model,capture_effect_thr,constant_per,pifs_activated,cw_adaptation,cont_wind,cont_wind_stage,throughput
306419,STA_H7,1,H,43.8570,13.9867,7.1174,2.4,6,4,4,...,12000,64,0,10,0.0,0,0,16,5,NaN
306420,STA_H8,1,H,33.9621,21.7909,5.6933,2.4,6,4,4,...,12000,64,0,10,0.0,0,0,16,5,NaN
306421,STA_H9,1,H,42.1158,13.5457,7.3812,2.4,6,4,4,...,12000,64,0,10,0.0,0,0,16,5,NaN
306422,STA_H10,1,H,40.0639,8.4542,4.6942,2.4,6,4,4,...,12000,64,0,10,0.0,0,0,16,5,NaN
306423,STA_H11,1,H,42.7649,14.7606,5.9192,2.4,6,4,4,...,12000,64,0,10,0.0,0,0,16,5,NaN


In [42]:
len(throughput)

1920

In [43]:
#remove nested list from throughput and convert it to a single list
throughput = [item for sublist in throughput for item in sublist]
# len(throughput)
rssi = [item for sublist in rssi for item in sublist]
sinr = [item for sublist in sinr for item in sublist]
# airtime = [item for sublist in airtime for item in sublist]



In [44]:
for idx,row in added_output_df.iterrows():
    added_output_df.at[idx,"throughput"] = throughput[idx]

In [45]:
added_output_df.head(16)

,node_code,node_type,wlan_code,x(m),y(m),z(m),central_freq(GHz),channel_bonding_model,primary_channel,min_channel_allowed,...,packet_length,num_packets_aggregated,capture_effect_model,capture_effect_thr,constant_per,pifs_activated,cw_adaptation,cont_wind,cont_wind_stage,throughput
0,AP_A,0,A,7.5000,8.3333,8.3417,5.0,4,0,0,...,12000,64,0,10,0.0,0,0,16,5,104.96
1,STA_A1,1,A,12.0627,4.6918,5.0670,5.0,4,0,0,...,12000,64,0,10,0.0,0,0,16,5,7.68
2,STA_A2,1,A,8.2712,4.8383,9.9676,5.0,4,0,0,...,12000,64,0,10,0.0,0,0,16,5,11.09
3,STA_A3,1,A,8.4824,3.5287,8.9153,5.0,4,0,0,...,12000,64,0,10,0.0,0,0,16,5,14.51
4,STA_A4,1,A,1.3639,1.8250,8.3635,5.0,4,0,0,...,12000,64,0,10,0.0,0,0,16,5,11.95
5,STA_A5,1,A,1.3786,3.2654,8.2911,5.0,4,0,0,...,12000,64,0,10,0.0,0,0,16,5,10.24
6,STA_A6,1,A,1.0444,1.2486,8.0232,5.0,4,0,0,...,12000,64,0,10,0.0,0,0,16,5,11.09
7,STA_A7,1,A,0.0853,13.7302,8.3594,5.0,4,0,0,...,12000,64,0,10,0.0,0,0,16,5,9.39
8,STA_A8,1,A,7.8169,18.2302,5.4278,5.0,4,0,0,...,12000,64,0,10,0.0,0,0,16,5,3.41
9,STA_A9,1,A,9.9062,5.0489,3.0879,5.0,4,0,0,...,12000,64,0,10,0.0,0,0,16,5,13.65


In [46]:
added_output_df["sinr"] = None

In [47]:
added_output_df["rssi"] = None
added_output_df["airtime"] = None

In [48]:
added_output_df.head(10)

,node_code,node_type,wlan_code,x(m),y(m),z(m),central_freq(GHz),channel_bonding_model,primary_channel,min_channel_allowed,...,capture_effect_thr,constant_per,pifs_activated,cw_adaptation,cont_wind,cont_wind_stage,throughput,sinr,rssi,airtime
0,AP_A,0,A,7.5000,8.3333,8.3417,5.0,4,0,0,...,10,0.0,0,0,16,5,104.96,None,None,None
1,STA_A1,1,A,12.0627,4.6918,5.0670,5.0,4,0,0,...,10,0.0,0,0,16,5,7.68,None,None,None
2,STA_A2,1,A,8.2712,4.8383,9.9676,5.0,4,0,0,...,10,0.0,0,0,16,5,11.09,None,None,None
3,STA_A3,1,A,8.4824,3.5287,8.9153,5.0,4,0,0,...,10,0.0,0,0,16,5,14.51,None,None,None
4,STA_A4,1,A,1.3639,1.8250,8.3635,5.0,4,0,0,...,10,0.0,0,0,16,5,11.95,None,None,None
5,STA_A5,1,A,1.3786,3.2654,8.2911,5.0,4,0,0,...,10,0.0,0,0,16,5,10.24,None,None,None
6,STA_A6,1,A,1.0444,1.2486,8.0232,5.0,4,0,0,...,10,0.0,0,0,16,5,11.09,None,None,None
7,STA_A7,1,A,0.0853,13.7302,8.3594,5.0,4,0,0,...,10,0.0,0,0,16,5,9.39,None,None,None
8,STA_A8,1,A,7.8169,18.2302,5.4278,5.0,4,0,0,...,10,0.0,0,0,16,5,3.41,None,None,None
9,STA_A9,1,A,9.9062,5.0489,3.0879,5.0,4,0,0,...,10,0.0,0,0,16,5,13.65,None,None,None


In [49]:
print(added_output_df["node_type"].value_counts())

node_type
1    287224
0     19200
Name: count, dtype: int64


In [50]:
len(sinr)

306424

In [51]:
for idx,row in added_output_df.iterrows():
    added_output_df.at[idx,"rssi"] = rssi[idx]
for idx,row in added_output_df.iterrows():
    added_output_df.at[idx,"sinr"] = sinr[idx]


In [53]:
count = -1
for idx,row in added_output_df.iterrows():
    if added_output_df.at[idx,"node_type"]==0:
        count+=1
        added_output_df.at[idx,"airtime"] = airtime[count]
    else:
        added_output_df.at[idx,"airtime"] = airtime[count]

In [ ]:
# count = -1
# for idx,row in added_output_df.iterrows():
#     if added_output_df.at[idx,"node_type"]==0:
#         count+=1
#         added_output_df.at[idx,"airtime"] = airtime[count]
#     else:
#         added_output_df.at[idx,"airtime"] = airtime[count]

In [54]:
interference_map[1]

[-76.64,
 inf,
 -75.96,
 -93.69,
 -85.25,
 -78.41,
 -85.97,
 -95.24,
 -100.63,
 -100.22,
 -100.5,
 -108.93]

In [55]:
added_output_df["int_map"] = None

In [56]:
count = -1
for idx,row in added_output_df.iterrows():
    if added_output_df.at[idx,"node_type"]==0:
        count+=1
        added_output_df.at[idx,"int_map"] = interference_map[count]
    else:
        added_output_df.at[idx,"int_map"] = interference_map[count]

In [13]:
added_output_df.head(14)

NameError: name 'added_output_df' is not defined

In [58]:
#signal features - edge features = rrsi,sinr , distance, interference (average)
#node features - airtime ,node_type, channel bonding model, primary channel - mainly airtime

In [59]:
added_output_df.to_csv("Combined.csv")

In [3]:
import pandas as pd
import numpy as np

In [30]:
df = pd.read_csv("Combined.csv")
df.head()

,Unnamed: 0,node_code,node_type,wlan_code,x(m),y(m),z(m),central_freq(GHz),channel_bonding_model,primary_channel,...,constant_per,pifs_activated,cw_adaptation,cont_wind,cont_wind_stage,throughput,sinr,rssi,airtime,int_map
0,0,AP_A,0,A,7.5000,8.3333,8.3417,5.0,4,0,...,0.0,0,0,16,5,104.96,inf,inf,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ..."
1,1,STA_A1,1,A,12.0627,4.6918,5.0670,5.0,4,0,...,0.0,0,0,16,5,7.68,26.99,-62.31,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ..."
2,2,STA_A2,1,A,8.2712,4.8383,9.9676,5.0,4,0,...,0.0,0,0,16,5,11.09,35.68,-55.42,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ..."
3,3,STA_A3,1,A,8.4824,3.5287,8.9153,5.0,4,0,...,0.0,0,0,16,5,14.51,33.42,-58.23,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ..."
4,4,STA_A4,1,A,1.3639,1.8250,8.3635,5.0,4,0,...,0.0,0,0,16,5,11.95,26.83,-66.64,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ..."


In [32]:
count = 0
flag = 0
sum = []
start_idx = 0
variance = 0
mean = 0
for idx,val in df.iterrows():
    if flag==1 and df.at[idx,"node_type"]==0:
        df.at[start_idx,"rssi"] = max(sum)
        flag = 0
        sum = []
        variance = 0
        count = 0

    if df.at[idx,"node_type"]==0:
        start_idx = idx
    else:
        sum.append(df["rssi"][idx])
        count+=1
        flag = 1

if flag == 1:
    df.at[start_idx, "rssi"] = max(sum)
    

In [33]:
df.head(15)

,Unnamed: 0,node_code,node_type,wlan_code,x(m),y(m),z(m),central_freq(GHz),channel_bonding_model,primary_channel,...,constant_per,pifs_activated,cw_adaptation,cont_wind,cont_wind_stage,throughput,sinr,rssi,airtime,int_map
0,0,AP_A,0,A,7.5000,8.3333,8.3417,5.0,4,0,...,0.0,0,0,16,5,104.96,inf,-55.42,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ..."
1,1,STA_A1,1,A,12.0627,4.6918,5.0670,5.0,4,0,...,0.0,0,0,16,5,7.68,26.99,-62.31,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ..."
2,2,STA_A2,1,A,8.2712,4.8383,9.9676,5.0,4,0,...,0.0,0,0,16,5,11.09,35.68,-55.42,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ..."
3,3,STA_A3,1,A,8.4824,3.5287,8.9153,5.0,4,0,...,0.0,0,0,16,5,14.51,33.42,-58.23,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ..."
4,4,STA_A4,1,A,1.3639,1.8250,8.3635,5.0,4,0,...,0.0,0,0,16,5,11.95,26.83,-66.64,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ..."
5,5,STA_A5,1,A,1.3786,3.2654,8.2911,5.0,4,0,...,0.0,0,0,16,5,10.24,27.90,-64.81,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ..."
6,6,STA_A6,1,A,1.0444,1.2486,8.0232,5.0,4,0,...,0.0,0,0,16,5,11.09,25.72,-67.76,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ..."
7,7,STA_A7,1,A,0.0853,13.7302,8.3594,5.0,4,0,...,0.0,0,0,16,5,9.39,23.29,-67.04,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ..."
8,8,STA_A8,1,A,7.8169,18.2302,5.4278,5.0,4,0,...,0.0,0,0,16,5,3.41,20.32,-68.98,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ..."
9,9,STA_A9,1,A,9.9062,5.0489,3.0879,5.0,4,0,...,0.0,0,0,16,5,13.65,27.57,-62.21,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ..."


In [104]:
df["sinr"].min()

-37.49

In [46]:
dist = pd.read_csv("distances_added.csv")

In [132]:
#convert the interference map string to a list with float values 
df["int_map"] = df["int_map"].apply(lambda x: x.strip('[]').split(','))
df["int_map"] = df["int_map"].apply(lambda x: [float(i) for i in x])

df.head()

,Unnamed: 0,node_code,node_type,wlan_code,x(m),y(m),z(m),central_freq(GHz),channel_bonding_model,primary_channel,...,constant_per,pifs_activated,cw_adaptation,cont_wind,cont_wind_stage,throughput,sinr,rssi,airtime,int_map
0,0,AP_A,0,A,7.5000,8.3333,8.3417,5.0,4,0,...,0.0,0,0,16,5,104.96,inf,-55.42,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ..."
1,1,STA_A1,1,A,12.0627,4.6918,5.0670,5.0,4,0,...,0.0,0,0,16,5,7.68,26.99,-62.31,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ..."
2,2,STA_A2,1,A,8.2712,4.8383,9.9676,5.0,4,0,...,0.0,0,0,16,5,11.09,35.68,-55.42,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ..."
3,3,STA_A3,1,A,8.4824,3.5287,8.9153,5.0,4,0,...,0.0,0,0,16,5,14.51,33.42,-58.23,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ..."
4,4,STA_A4,1,A,1.3639,1.8250,8.3635,5.0,4,0,...,0.0,0,0,16,5,11.95,26.83,-66.64,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ..."


In [139]:
#replace all infinity values with 0 in interference map
for idx,row in df.iterrows():
    for i in range(len(df.at[idx,"int_map"])):
        if df.at[idx,"int_map"][i]==float('inf'):
            df.at[idx,"int_map"][i]=0

# df.head(15)

,Unnamed: 0,node_code,node_type,wlan_code,x(m),y(m),z(m),central_freq(GHz),channel_bonding_model,primary_channel,...,constant_per,pifs_activated,cw_adaptation,cont_wind,cont_wind_stage,throughput,sinr,rssi,airtime,int_map
0,0,AP_A,0,A,7.5000,8.3333,8.3417,5.0,4,0,...,0.0,0,0,16,5,104.96,inf,-55.42,"[50.3, 50.3, 50.3, 50.3]","[0, -79.65, -93.86, -109.01, -78.68, -85.3, -9..."
1,1,STA_A1,1,A,12.0627,4.6918,5.0670,5.0,4,0,...,0.0,0,0,16,5,7.68,26.99,-62.31,"[50.3, 50.3, 50.3, 50.3]","[0, -79.65, -93.86, -109.01, -78.68, -85.3, -9..."
2,2,STA_A2,1,A,8.2712,4.8383,9.9676,5.0,4,0,...,0.0,0,0,16,5,11.09,35.68,-55.42,"[50.3, 50.3, 50.3, 50.3]","[0, -79.65, -93.86, -109.01, -78.68, -85.3, -9..."
3,3,STA_A3,1,A,8.4824,3.5287,8.9153,5.0,4,0,...,0.0,0,0,16,5,14.51,33.42,-58.23,"[50.3, 50.3, 50.3, 50.3]","[0, -79.65, -93.86, -109.01, -78.68, -85.3, -9..."
4,4,STA_A4,1,A,1.3639,1.8250,8.3635,5.0,4,0,...,0.0,0,0,16,5,11.95,26.83,-66.64,"[50.3, 50.3, 50.3, 50.3]","[0, -79.65, -93.86, -109.01, -78.68, -85.3, -9..."
5,5,STA_A5,1,A,1.3786,3.2654,8.2911,5.0,4,0,...,0.0,0,0,16,5,10.24,27.90,-64.81,"[50.3, 50.3, 50.3, 50.3]","[0, -79.65, -93.86, -109.01, -78.68, -85.3, -9..."
6,6,STA_A6,1,A,1.0444,1.2486,8.0232,5.0,4,0,...,0.0,0,0,16,5,11.09,25.72,-67.76,"[50.3, 50.3, 50.3, 50.3]","[0, -79.65, -93.86, -109.01, -78.68, -85.3, -9..."
7,7,STA_A7,1,A,0.0853,13.7302,8.3594,5.0,4,0,...,0.0,0,0,16,5,9.39,23.29,-67.04,"[50.3, 50.3, 50.3, 50.3]","[0, -79.65, -93.86, -109.01, -78.68, -85.3, -9..."
8,8,STA_A8,1,A,7.8169,18.2302,5.4278,5.0,4,0,...,0.0,0,0,16,5,3.41,20.32,-68.98,"[50.3, 50.3, 50.3, 50.3]","[0, -79.65, -93.86, -109.01, -78.68, -85.3, -9..."
9,9,STA_A9,1,A,9.9062,5.0489,3.0879,5.0,4,0,...,0.0,0,0,16,5,13.65,27.57,-62.21,"[50.3, 50.3, 50.3, 50.3]","[0, -79.65, -93.86, -109.01, -78.68, -85.3, -9..."


In [32]:
flag = 0

start_idx = 0
variance = 0
mean = 0
val_rssi = df.at[start_idx,"rssi"] 
for idx,val in df.iterrows():
    if flag==1 and df.at[idx,"node_type"]==0:
        df.at[start_idx, "sinr"] = sinr
        val_rssi = df.at[idx,"rssi"] 
        flag = 0
        sinr = 0

    if df.at[idx,"node_type"]==0:
        start_idx = idx
    elif df["rssi"][idx]==val_rssi and df.at[idx,"node_type"]==1:
        sinr = df["sinr"][idx]
        flag = 1

if flag == 1:
    df.at[start_idx, "sinr"] = sinr
df.head(15)
        

    
    

,Unnamed: 0.1,Unnamed: 0,node_code,node_type,wlan_code,x(m),y(m),z(m),central_freq(GHz),channel_bonding_model,...,constant_per,pifs_activated,cw_adaptation,cont_wind,cont_wind_stage,throughput,sinr,rssi,airtime,int_map
0,0,0,AP_A,0,A,7.5000,8.3333,8.3417,5.0,4,...,0.0,0,0,16,5,104.96,35.68,-55.42,"[50.3, 50.3, 50.3, 50.3]","[0, -79.65, -93.86, -109.01, -78.68, -85.3, -9..."
1,1,1,STA_A1,1,A,12.0627,4.6918,5.0670,5.0,4,...,0.0,0,0,16,5,7.68,26.99,-62.31,"[50.3, 50.3, 50.3, 50.3]","[0, -79.65, -93.86, -109.01, -78.68, -85.3, -9..."
2,2,2,STA_A2,1,A,8.2712,4.8383,9.9676,5.0,4,...,0.0,0,0,16,5,11.09,35.68,-55.42,"[50.3, 50.3, 50.3, 50.3]","[0, -79.65, -93.86, -109.01, -78.68, -85.3, -9..."
3,3,3,STA_A3,1,A,8.4824,3.5287,8.9153,5.0,4,...,0.0,0,0,16,5,14.51,33.42,-58.23,"[50.3, 50.3, 50.3, 50.3]","[0, -79.65, -93.86, -109.01, -78.68, -85.3, -9..."
4,4,4,STA_A4,1,A,1.3639,1.8250,8.3635,5.0,4,...,0.0,0,0,16,5,11.95,26.83,-66.64,"[50.3, 50.3, 50.3, 50.3]","[0, -79.65, -93.86, -109.01, -78.68, -85.3, -9..."
5,5,5,STA_A5,1,A,1.3786,3.2654,8.2911,5.0,4,...,0.0,0,0,16,5,10.24,27.90,-64.81,"[50.3, 50.3, 50.3, 50.3]","[0, -79.65, -93.86, -109.01, -78.68, -85.3, -9..."
6,6,6,STA_A6,1,A,1.0444,1.2486,8.0232,5.0,4,...,0.0,0,0,16,5,11.09,25.72,-67.76,"[50.3, 50.3, 50.3, 50.3]","[0, -79.65, -93.86, -109.01, -78.68, -85.3, -9..."
7,7,7,STA_A7,1,A,0.0853,13.7302,8.3594,5.0,4,...,0.0,0,0,16,5,9.39,23.29,-67.04,"[50.3, 50.3, 50.3, 50.3]","[0, -79.65, -93.86, -109.01, -78.68, -85.3, -9..."
8,8,8,STA_A8,1,A,7.8169,18.2302,5.4278,5.0,4,...,0.0,0,0,16,5,3.41,20.32,-68.98,"[50.3, 50.3, 50.3, 50.3]","[0, -79.65, -93.86, -109.01, -78.68, -85.3, -9..."
9,9,9,STA_A9,1,A,9.9062,5.0489,3.0879,5.0,4,...,0.0,0,0,16,5,13.65,27.57,-62.21,"[50.3, 50.3, 50.3, 50.3]","[0, -79.65, -93.86, -109.01, -78.68, -85.3, -9..."


In [33]:
df.to_csv("Final_dataset.csv")

# IMPUTING SINR

In [34]:
import pandas as pd 
import numpy as np

In [35]:
df = pd.read_csv("Final_dataset.csv")
df.isna().sum()

Unnamed: 0.2                   0
Unnamed: 0.1                   0
Unnamed: 0                     0
node_code                      0
node_type                      0
wlan_code                      0
x(m)                           0
y(m)                           0
z(m)                           0
central_freq(GHz)              0
channel_bonding_model          0
primary_channel                0
min_channel_allowed            0
max_channel_allowed            0
tpc_default(dBm)               0
cca_default(dBm)               0
traffic_model                  0
traffic_load[pkt/s]            0
packet_length                  0
num_packets_aggregated         0
capture_effect_model           0
capture_effect_thr             0
constant_per                   0
pifs_activated                 0
cw_adaptation                  0
cont_wind                      0
cont_wind_stage                0
throughput                     0
sinr                      119604
rssi                           0
airtime   

In [37]:
df["sinr"][11]

31.29

In [38]:
for idx,val in df.iterrows():
    if df.at[idx,"sinr"] == np.inf:
        print(idx)

In [39]:
pd.isna(df["sinr"][306422])

True

## 2 DATAFRAMES FROM HERE

In [47]:
#choice 1 - if at least one sinr is -ve and not nan, assign all nodes to that sinr else 0 to -20
import random
# print(sinr_temp)
for idx,val in df.iterrows():
    if df.at[idx,"node_type"]==0:
        sinr_temp = round(random.uniform(0,-20),2)
        j = idx
        sinr = df.at[idx,"sinr"]
        while(df.at[j,"node_type"]!=0 and j!=df.shape[0]):
            if pd.isna(df.at[j,"sinr"]):
                j+=1
                continue
            if df.at[j,"sinr"]<0:
                sinr = df.at[idx,"sinr"]
            j+=1
    if pd.isna(df.at[idx,"sinr"]):    
        df.at[idx,"sinr"] = sinr_temp
    

In [49]:
# df[306390:306430]
df.isna().sum()

Unnamed: 0.2              0
Unnamed: 0.1              0
Unnamed: 0                0
node_code                 0
node_type                 0
wlan_code                 0
x(m)                      0
y(m)                      0
z(m)                      0
central_freq(GHz)         0
channel_bonding_model     0
primary_channel           0
min_channel_allowed       0
max_channel_allowed       0
tpc_default(dBm)          0
cca_default(dBm)          0
traffic_model             0
traffic_load[pkt/s]       0
packet_length             0
num_packets_aggregated    0
capture_effect_model      0
capture_effect_thr        0
constant_per              0
pifs_activated            0
cw_adaptation             0
cont_wind                 0
cont_wind_stage           0
throughput                0
sinr                      0
rssi                      0
airtime                   0
int_map                   0
dtype: int64

In [35]:
# sinr = 999999

# for idx,val in df.iterrows():
#     if df.at[idx,"node_type"]==0:
#         j = idx
#         sinr = df.at[idx,"sinr"]
#         while(df.at[j,"node_type"]!=0 and j!=df.shape[0]):
#             if pd.isna(df.at[j,"sinr"]):
#                 j+=1
#                 continue
#             sinr = min(sinr,df.at[j,"sinr"])
#             j+=1
    
#     elif df.at[idx,"node_type"]==1 and pd.isna(df.at[idx,"sinr"]):
#         df.at[idx,"sinr"] = sinr

            
    

In [50]:
dist = pd.read_csv("distances_added.csv")

In [54]:
# dist = pd.read_csv("distances_added.csv")
new_df = df.copy()
cols_to_add = ["distance","distance_mean","distance_std"]
for col in cols_to_add:
    new_df[col] = dist[col]

# new_df.head()

In [55]:
new_df.head(2)

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,node_code,node_type,wlan_code,x(m),y(m),z(m),central_freq(GHz),...,cont_wind,cont_wind_stage,throughput,sinr,rssi,airtime,int_map,distance,distance_mean,distance_std
0,0,0,0,AP_A,0,A,7.5000,8.3333,8.3417,5.0,...,16,5,104.96,35.68,-55.42,"[50.3, 50.3, 50.3, 50.3]","[0, -79.65, -93.86, -109.01, -78.68, -85.3, -9...",NaN,7.816307,8.088183
1,1,1,1,STA_A1,1,A,12.0627,4.6918,5.0670,5.0,...,16,5,7.68,26.99,-62.31,"[50.3, 50.3, 50.3, 50.3]","[0, -79.65, -93.86, -109.01, -78.68, -85.3, -9...",6.693461,NaN,NaN


In [ ]:
import math
count = 0
flag = 0
sum =0
start_idx = 0
variance = 0
mean = 0
for idx,val in new_df.iterrows():
    if flag==1 and new_df.at[idx,"node_type"]==0:
        new_df.at[start_idx,"sinr_mean"] = sum/count
        new_df.at[start_idx, "sinr_std"] = math.sqrt(variance / count)
        flag = 0
        sum = 0
        variance = 0
        count = 0

    if new_df.at[idx,"node_type"]==0:
        start_idx = idx
    else:
        sum+=new_df["sinr"][idx]
        variance += (new_df["sinr"][idx] - mean) ** 2
        count+=1
        flag = 1

if flag == 1:
    mean = sum / count
    new_df.at[start_idx, "sinr_mean"] = mean
    new_df.at[start_idx, "sinr_std"] = math.sqrt(variance / count)
    

In [63]:
import math
count = 0
flag = 0
sum =0
start_idx = 0
variance = 0
mean = 0
for idx,val in new_df.iterrows():
    if flag==1 and new_df.at[idx,"node_type"]==0:
        new_df.at[start_idx,"rssi_mean"] = sum/count
        new_df.at[start_idx, "rssi_std"] = math.sqrt(variance / count)
        flag = 0
        sum = 0
        variance = 0
        count = 0

    if new_df.at[idx,"node_type"]==0:
        start_idx = idx
    else:
        sum+=new_df["rssi"][idx]
        variance += (new_df["rssi"][idx] - mean) ** 2
        count+=1
        flag = 1

if flag == 1:
    mean = sum / count
    new_df.at[start_idx, "rssi_mean"] = mean
    new_df.at[start_idx, "rssi_std"] = math.sqrt(variance / count)
    

In [67]:
#get the number of stations under each AP
count = 0
flag = 0
sum =0
start_idx = 0
variance = 0
mean = 0
for idx,val in new_df.iterrows():
    if flag==1 and new_df.at[idx,"node_type"]==0:
        new_df.at[start_idx,"num_stations"] = count
        flag = 0
        count = 0

    if new_df.at[idx,"node_type"]==0:
        start_idx = idx
    else:
        count+=1
        flag = 1

if flag == 1:
    num_stations = count
    new_df.at[start_idx, "num_stations"] = count
    

In [71]:
for idx,val in enumerate(new_df["airtime"]):
    for i in range(len(eval(val))):
        new_df.at[idx,"airtime_"+str(i)] = eval(val)[i]

In [72]:
new_df.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,node_code,node_type,wlan_code,x(m),y(m),z(m),central_freq(GHz),...,rssi_std,num_stations,airtime_0,airtime_1,airtime_2,airtime_3,airtime_4,airtime_5,airtime_6,airtime_7
0,0,0,0,AP_A,0,A,7.5000,8.3333,8.3417,5.0,...,64.328276,10.0,50.3,50.3,50.3,50.3,NaN,NaN,NaN,NaN
1,1,1,1,STA_A1,1,A,12.0627,4.6918,5.0670,5.0,...,NaN,NaN,50.3,50.3,50.3,50.3,NaN,NaN,NaN,NaN
2,2,2,2,STA_A2,1,A,8.2712,4.8383,9.9676,5.0,...,NaN,NaN,50.3,50.3,50.3,50.3,NaN,NaN,NaN,NaN
3,3,3,3,STA_A3,1,A,8.4824,3.5287,8.9153,5.0,...,NaN,NaN,50.3,50.3,50.3,50.3,NaN,NaN,NaN,NaN
4,4,4,4,STA_A4,1,A,1.3639,1.8250,8.3635,5.0,...,NaN,NaN,50.3,50.3,50.3,50.3,NaN,NaN,NaN,NaN


In [73]:
for idx,val in new_df.iterrows():
    if new_df.at[idx,"min_channel_allowed"]==2 and new_df.at[idx,"node_type"]==0:
        if len(new_df.at[idx,"airtime"].split())!=(new_df.at[idx,"max_channel_allowed"]-new_df.at[idx,"min_channel_allowed"]+1):
            print(idx)
            print(new_df["min_channel_allowed"][idx])
            print(new_df["max_channel_allowed"][idx])
            print(new_df["airtime"][idx])

In [75]:
new_df.columns

Index(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'node_code', 'node_type',
       'wlan_code', 'x(m)', 'y(m)', 'z(m)', 'central_freq(GHz)',
       'channel_bonding_model', 'primary_channel', 'min_channel_allowed',
       'max_channel_allowed', 'tpc_default(dBm)', 'cca_default(dBm)',
       'traffic_model', 'traffic_load[pkt/s]', 'packet_length',
       'num_packets_aggregated', 'capture_effect_model', 'capture_effect_thr',
       'constant_per', 'pifs_activated', 'cw_adaptation', 'cont_wind',
       'cont_wind_stage', 'throughput', 'sinr', 'rssi', 'airtime', 'int_map',
       'distance', 'distance_mean', 'distance_std', 'sinr_mean', 'sinr_std',
       'rssi_mean', 'rssi_std', 'num_stations', 'airtime_0', 'airtime_1',
       'airtime_2', 'airtime_3', 'airtime_4', 'airtime_5', 'airtime_6',
       'airtime_7'],
      dtype='object')

In [76]:
new_df.drop(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0'],axis=1,inplace=True)

In [79]:
new_df.to_csv("RAMON_DATASET.csv",index=False)

In [82]:
df = pd.read_csv("RAMON_DATASET.csv")
df[20:40]

,node_code,node_type,wlan_code,x(m),y(m),z(m),central_freq(GHz),channel_bonding_model,primary_channel,min_channel_allowed,...,rssi_std,num_stations,airtime_0,airtime_1,airtime_2,airtime_3,airtime_4,airtime_5,airtime_6,airtime_7
20,STA_B9,1,B,15.4478,15.3003,4.1624,5.0,4,0,0,...,NaN,NaN,24.50,24.50,24.50,24.50,24.5,24.5,24.5,24.5
21,STA_B10,1,B,26.9212,4.4209,3.8776,5.0,4,0,0,...,NaN,NaN,24.50,24.50,24.50,24.50,24.5,24.5,24.5,24.5
22,STA_B11,1,B,19.0803,17.4127,4.8268,5.0,4,0,0,...,NaN,NaN,24.50,24.50,24.50,24.50,24.5,24.5,24.5,24.5
23,STA_B12,1,B,21.3354,16.5556,6.8528,5.0,4,0,0,...,NaN,NaN,24.50,24.50,24.50,24.50,24.5,24.5,24.5,24.5
24,STA_B13,1,B,20.9281,-0.6885,6.2825,5.0,4,0,0,...,NaN,NaN,24.50,24.50,24.50,24.50,24.5,24.5,24.5,24.5
25,STA_B14,1,B,28.7108,0.5157,9.4268,5.0,4,0,0,...,NaN,NaN,24.50,24.50,24.50,24.50,24.5,24.5,24.5,24.5
26,STA_B15,1,B,28.8785,13.3031,3.6122,5.0,4,0,0,...,NaN,NaN,24.50,24.50,24.50,24.50,24.5,24.5,24.5,24.5
27,AP_C,0,C,37.5000,8.3333,5.1559,5.0,4,0,0,...,62.937206,16.0,26.46,19.73,19.73,19.73,NaN,NaN,NaN,NaN
28,STA_C1,1,C,42.0886,13.2795,9.9051,5.0,4,0,0,...,NaN,NaN,26.46,19.73,19.73,19.73,NaN,NaN,NaN,NaN
29,STA_C2,1,C,36.1528,0.2123,8.4019,5.0,4,0,0,...,NaN,NaN,26.46,19.73,19.73,19.73,NaN,NaN,NaN,NaN


In [83]:
print(df["min_channel_allowed"][27])
print(df["max_channel_allowed"][27])


0
3


In [86]:
for idx,val in df.iterrows():
    if pd.isna(df.at[idx,"airtime_0"]):
        df.at[idx,"airtime_0"] = 0
    if pd.isna(df.at[idx,"airtime_1"]):
        df.at[idx,"airtime_1"] = 0
    if pd.isna(df.at[idx,"airtime_2"]):
        df.at[idx,"airtime_2"] = 0
    if pd.isna(df.at[idx,"airtime_3"]):
        df.at[idx,"airtime_3"] = 0
    if pd.isna(df.at[idx,"airtime_4"]):
        df.at[idx,"airtime_4"] = 0
    if pd.isna(df.at[idx,"airtime_5"]):
        df.at[idx,"airtime_5"] = 0
    if pd.isna(df.at[idx,"airtime_6"]):
        df.at[idx,"airtime_6"] = 0
    if pd.isna(df.at[idx,"airtime_7"]):
        df.at[idx,"airtime_7"] = 0

In [87]:
df.head()

,node_code,node_type,wlan_code,x(m),y(m),z(m),central_freq(GHz),channel_bonding_model,primary_channel,min_channel_allowed,...,rssi_std,num_stations,airtime_0,airtime_1,airtime_2,airtime_3,airtime_4,airtime_5,airtime_6,airtime_7
0,AP_A,0,A,7.5000,8.3333,8.3417,5.0,4,0,0,...,64.328276,10.0,50.3,50.3,50.3,50.3,0.0,0.0,0.0,0.0
1,STA_A1,1,A,12.0627,4.6918,5.0670,5.0,4,0,0,...,NaN,NaN,50.3,50.3,50.3,50.3,0.0,0.0,0.0,0.0
2,STA_A2,1,A,8.2712,4.8383,9.9676,5.0,4,0,0,...,NaN,NaN,50.3,50.3,50.3,50.3,0.0,0.0,0.0,0.0
3,STA_A3,1,A,8.4824,3.5287,8.9153,5.0,4,0,0,...,NaN,NaN,50.3,50.3,50.3,50.3,0.0,0.0,0.0,0.0
4,STA_A4,1,A,1.3639,1.8250,8.3635,5.0,4,0,0,...,NaN,NaN,50.3,50.3,50.3,50.3,0.0,0.0,0.0,0.0


In [88]:
df.isna().sum()

node_code                      0
node_type                      0
wlan_code                      0
x(m)                           0
y(m)                           0
z(m)                           0
central_freq(GHz)              0
channel_bonding_model          0
primary_channel                0
min_channel_allowed            0
max_channel_allowed            0
tpc_default(dBm)               0
cca_default(dBm)               0
traffic_model                  0
traffic_load[pkt/s]            0
packet_length                  0
num_packets_aggregated         0
capture_effect_model           0
capture_effect_thr             0
constant_per                   0
pifs_activated                 0
cw_adaptation                  0
cont_wind                      0
cont_wind_stage                0
throughput                     0
sinr                           0
rssi                           0
airtime                        0
int_map                        0
distance                   19200
distance_m

In [89]:
df.to_csv("RAMON_DATASET.csv")

# RAMON

In [ ]:
import torchvision
import numpy as np
import pandas as pd

In [ ]:
df= pd.read_csv("Combined.csv")
df.head()

,Unnamed: 0,node_code,node_type,wlan_code,x(m),y(m),z(m),central_freq(GHz),channel_bonding_model,primary_channel,...,constant_per,pifs_activated,cw_adaptation,cont_wind,cont_wind_stage,throughput,sinr,rssi,airtime,int_map
0,0,AP_A,0,A,7.5000,8.3333,8.3417,5.0,4,0,...,0.0,0,0,16,5,104.96,inf,inf,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ..."
1,1,STA_A1,1,A,12.0627,4.6918,5.0670,5.0,4,0,...,0.0,0,0,16,5,7.68,26.99,-62.31,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ..."
2,2,STA_A2,1,A,8.2712,4.8383,9.9676,5.0,4,0,...,0.0,0,0,16,5,11.09,35.68,-55.42,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ..."
3,3,STA_A3,1,A,8.4824,3.5287,8.9153,5.0,4,0,...,0.0,0,0,16,5,14.51,33.42,-58.23,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ..."
4,4,STA_A4,1,A,1.3639,1.8250,8.3635,5.0,4,0,...,0.0,0,0,16,5,11.95,26.83,-66.64,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ..."


In [ ]:
# np.mean(df["sinr"])

In [ ]:
df["sinr"] = df['sinr'].replace(np.inf,0)

In [ ]:
df["sinr_mean"] = None

In [ ]:
import math

In [ ]:
count = 0
flag = 0
sum =0
start_idx = 0
variance = 0
mean = 0
for idx,val in df.iterrows():
    if flag==1 and df.at[idx,"node_type"]==0:
        df.at[start_idx,"sinr_mean"] = sum/count
        df.at[start_idx, "sinr_std"] = math.sqrt(variance / count)
        flag = 0
        sum = 0
        variance = 0
        count = 0

    if df.at[idx,"node_type"]==0:
        start_idx = idx
    else:
        sum+=df["sinr"][idx]
        variance += (df["sinr"][idx] - mean) ** 2
        count+=1
        flag = 1

if flag == 1:
    mean = sum / count
    df.at[start_idx, "sinr_mean"] = mean
    df.at[start_idx, "sinr_std"] = math.sqrt(variance / count)
    

In [ ]:
#rssi
count = 0
flag = 0
sum =0
start_idx = 0
variance = 0
mean = 0
for idx,val in df.iterrows():
    if flag==1 and df.at[idx,"node_type"]==0:
        df.at[start_idx,"rssi_mean"] = sum/count
        df.at[start_idx, "rssi_std"] = math.sqrt(variance / count)
        flag = 0
        sum = 0
        variance = 0
        count = 0

    if df.at[idx,"node_type"]==0:
        start_idx = idx
    else:
        sum+=df["rssi"][idx]
        variance += (df["rssi"][idx] - mean) ** 2
        count+=1
        flag = 1

if flag == 1:
    mean = sum / count
    df.at[start_idx, "rssi_mean"] = mean
    df.at[start_idx, "rssi_std"] = math.sqrt(variance / count)
    

In [ ]:
df.head()

,Unnamed: 0,node_code,node_type,wlan_code,x(m),y(m),z(m),central_freq(GHz),channel_bonding_model,primary_channel,...,cont_wind_stage,throughput,sinr,rssi,airtime,int_map,sinr_mean,sinr_std,rssi_mean,rssi_std
0,0,AP_A,0,A,7.5000,8.3333,8.3417,5.0,4,0,...,5,104.96,0.00,inf,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",27.009,27.375253,-64.182,64.328276
1,1,STA_A1,1,A,12.0627,4.6918,5.0670,5.0,4,0,...,5,7.68,26.99,-62.31,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",None,NaN,NaN,NaN
2,2,STA_A2,1,A,8.2712,4.8383,9.9676,5.0,4,0,...,5,11.09,35.68,-55.42,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",None,NaN,NaN,NaN
3,3,STA_A3,1,A,8.4824,3.5287,8.9153,5.0,4,0,...,5,14.51,33.42,-58.23,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",None,NaN,NaN,NaN
4,4,STA_A4,1,A,1.3639,1.8250,8.3635,5.0,4,0,...,5,11.95,26.83,-66.64,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",None,NaN,NaN,NaN


In [ ]:
#find the range of values in rssi column
max_rssi = df["rssi"].max()
min_rssi = df["rssi"].min()
max_rssi,min_rssi

(inf, -74.68)

In [ ]:
df.head()

,Unnamed: 0,node_code,node_type,wlan_code,x(m),y(m),z(m),central_freq(GHz),channel_bonding_model,primary_channel,...,cont_wind_stage,throughput,sinr,rssi,airtime,int_map,sinr_mean,sinr_std,rssi_mean,rssi_std
0,0,AP_A,0,A,7.5000,8.3333,8.3417,5.0,4,0,...,5,104.96,0.00,inf,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",27.009,27.375253,-64.182,64.328276
1,1,STA_A1,1,A,12.0627,4.6918,5.0670,5.0,4,0,...,5,7.68,26.99,-62.31,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",None,NaN,NaN,NaN
2,2,STA_A2,1,A,8.2712,4.8383,9.9676,5.0,4,0,...,5,11.09,35.68,-55.42,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",None,NaN,NaN,NaN
3,3,STA_A3,1,A,8.4824,3.5287,8.9153,5.0,4,0,...,5,14.51,33.42,-58.23,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",None,NaN,NaN,NaN
4,4,STA_A4,1,A,1.3639,1.8250,8.3635,5.0,4,0,...,5,11.95,26.83,-66.64,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",None,NaN,NaN,NaN


In [ ]:
#replace rssi values with 10000 as a random float number between -55 and -74 truncate to 2 decimal places


import random
for idx,val in df.iterrows():
    if df.at[idx,"node_type"]==0:
        df.at[idx,"rssi"] = round(random.uniform(-10,-20),2)



In [ ]:
df.to_csv("Combined_1.0.csv")

In [ ]:
for idx,val in df.iterrows():
    #calculate distance between the access point and the station 
    if df.at[idx,"node_type"]==0:
        ap_x,ap_y,ap_z = df.at[idx,"x(m)"],df.at[idx,"y(m)"],df.at[idx,"z(m)"]
    else:
        df.at[idx,"distance"] = math.sqrt((ap_x-df.at[idx,"x(m)"])**2 + (ap_y-df.at[idx,"y(m)"])**2 + (ap_z-df.at[idx,"z(m)"])**2)
        

In [ ]:
df.head()

,Unnamed: 0,node_code,node_type,wlan_code,x(m),y(m),z(m),central_freq(GHz),channel_bonding_model,primary_channel,...,throughput,sinr,rssi,airtime,int_map,sinr_mean,sinr_std,rssi_mean,rssi_std,distance
0,0,AP_A,0,A,7.5000,8.3333,8.3417,5.0,4,0,...,104.96,0.00,-10.68,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",27.009,27.375253,-64.182,64.328276,NaN
1,1,STA_A1,1,A,12.0627,4.6918,5.0670,5.0,4,0,...,7.68,26.99,-62.31,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",None,NaN,NaN,NaN,6.693461
2,2,STA_A2,1,A,8.2712,4.8383,9.9676,5.0,4,0,...,11.09,35.68,-55.42,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",None,NaN,NaN,NaN,3.931072
3,3,STA_A3,1,A,8.4824,3.5287,8.9153,5.0,4,0,...,14.51,33.42,-58.23,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",None,NaN,NaN,NaN,4.937439
4,4,STA_A4,1,A,1.3639,1.8250,8.3635,5.0,4,0,...,11.95,26.83,-66.64,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",None,NaN,NaN,NaN,8.944840


In [ ]:
count = 0
flag = 0
sum =0
start_idx = 0
variance = 0
mean = 0
for idx,val in df.iterrows():
    if flag==1 and df.at[idx,"node_type"]==0:
        df.at[start_idx,"distance_mean"] = sum/count
        df.at[start_idx, "distance_std"] = math.sqrt(variance / count)
        flag = 0
        sum = 0
        variance = 0
        count = 0

    if df.at[idx,"node_type"]==0:
        start_idx = idx
    else:
        sum+=df["distance"][idx]
        variance += (df["distance"][idx] - mean) ** 2
        count+=1
        flag = 1

if flag == 1:
    mean = sum / count
    df.at[start_idx, "distance_mean"] = mean
    df.at[start_idx, "distance_std"] = math.sqrt(variance / count)
    

In [ ]:
df.head(14)

,Unnamed: 0,node_code,node_type,wlan_code,x(m),y(m),z(m),central_freq(GHz),channel_bonding_model,primary_channel,...,rssi,airtime,int_map,sinr_mean,sinr_std,rssi_mean,rssi_std,distance,distance_mean,distance_std
0,0,AP_A,0,A,7.5000,8.3333,8.3417,5.0,4,0,...,-10.68,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",27.009,27.375253,-64.182000,64.328276,NaN,7.816307,8.088183
1,1,STA_A1,1,A,12.0627,4.6918,5.0670,5.0,4,0,...,-62.31,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",None,NaN,NaN,NaN,6.693461,NaN,NaN
2,2,STA_A2,1,A,8.2712,4.8383,9.9676,5.0,4,0,...,-55.42,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",None,NaN,NaN,NaN,3.931072,NaN,NaN
3,3,STA_A3,1,A,8.4824,3.5287,8.9153,5.0,4,0,...,-58.23,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",None,NaN,NaN,NaN,4.937439,NaN,NaN
4,4,STA_A4,1,A,1.3639,1.8250,8.3635,5.0,4,0,...,-66.64,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",None,NaN,NaN,NaN,8.944840,NaN,NaN
5,5,STA_A5,1,A,1.3786,3.2654,8.2911,5.0,4,0,...,-64.81,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",None,NaN,NaN,NaN,7.947182,NaN,NaN
6,6,STA_A6,1,A,1.0444,1.2486,8.0232,5.0,4,0,...,-67.76,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",None,NaN,NaN,NaN,9.590057,NaN,NaN
7,7,STA_A7,1,A,0.0853,13.7302,8.3594,5.0,4,0,...,-67.04,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",None,NaN,NaN,NaN,9.170857,NaN,NaN
8,8,STA_A8,1,A,7.8169,18.2302,5.4278,5.0,4,0,...,-68.98,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",None,NaN,NaN,NaN,10.321815,NaN,NaN
9,9,STA_A9,1,A,9.9062,5.0489,3.0879,5.0,4,0,...,-62.21,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",None,NaN,NaN,NaN,6.646766,NaN,NaN


In [ ]:
#NaN values in the sinr column
#padding 0s in accordance to min channel
#interference sources
#variability

In [ ]:
df.to_csv("distances_added.csv")

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("distances_added.csv")
df.head()

,Unnamed: 0.1,Unnamed: 0,node_code,node_type,wlan_code,x(m),y(m),z(m),central_freq(GHz),channel_bonding_model,...,rssi,airtime,int_map,sinr_mean,sinr_std,rssi_mean,rssi_std,distance,distance_mean,distance_std
0,0,0,AP_A,0,A,7.5000,8.3333,8.3417,5.0,4,...,-10.68,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",27.009,27.375253,-64.182,64.328276,NaN,7.816307,8.088183
1,1,1,STA_A1,1,A,12.0627,4.6918,5.0670,5.0,4,...,-62.31,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",NaN,NaN,NaN,NaN,6.693461,NaN,NaN
2,2,2,STA_A2,1,A,8.2712,4.8383,9.9676,5.0,4,...,-55.42,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",NaN,NaN,NaN,NaN,3.931072,NaN,NaN
3,3,3,STA_A3,1,A,8.4824,3.5287,8.9153,5.0,4,...,-58.23,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",NaN,NaN,NaN,NaN,4.937439,NaN,NaN
4,4,4,STA_A4,1,A,1.3639,1.8250,8.3635,5.0,4,...,-66.64,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",NaN,NaN,NaN,NaN,8.944840,NaN,NaN


In [ ]:
#get the number of stations under each AP
count = 0
flag = 0
sum =0
start_idx = 0
variance = 0
mean = 0
for idx,val in df.iterrows():
    if flag==1 and df.at[idx,"node_type"]==0:
        df.at[start_idx,"num_stations"] = count
        flag = 0
        count = 0

    if df.at[idx,"node_type"]==0:
        start_idx = idx
    else:
        count+=1
        flag = 1

if flag == 1:
    num_stations = count
    df.at[start_idx, "num_stations"] = count
    

In [ ]:
for idx,val in df.iterrows():
    if df.at[idx,"min_channel_allowed"]==2 and df.at[idx,"node_type"]==0:
        if len(df.at[idx,"airtime"].split())!=(df.at[idx,"max_channel_allowed"]-df.at[idx,"min_channel_allowed"]+1):
            print(idx)
            print(df["min_channel_allowed"][idx])
            print(df["max_channel_allowed"][idx])
            print(df["airtime"][idx])

188
2
3
[89.44, 74.63, 45.07, 45.07]
241
2
3
[70.84, 70.84, 70.84, 70.84, 46.41, 46.41, 46.41, 46.41]
276
2
3
[37.2, 26.73, 7.09, 7.09]
308
2
3
[19.28, 13.52, 13.51, 13.51, 5.13, 5.13, 5.13, 5.13]
405
2
3
[]
887
2
3
[77.82, 36.8, 32.56, 32.56]
1109
2
3
[45.96, 15.05, 15.05, 15.05]
1331
2
3
[14.98, 10.95, 10.95, 10.95, 10.22, 10.22, 10.22, 10.22]
1347
2
3
[1.59, 1.59, 1.59, 1.59, 1.59, 1.59, 1.59, 1.59]
1360
2
3
[66.91, 35.66, 30.31, 30.31, 16.82, 16.82, 16.82, 16.82]
1373
2
3
[85.47, 81.44, 81.44, 81.44]
1441
2
3
[]
1496
2
3
[89.73, 89.73, 61.76, 61.76, 41.06, 41.06, 41.06, 41.06]
1832
2
3
[52.77, 46.21, 46.21, 46.21]
2373
2
3
[0.21, 0.21, 0.21, 0.21, 0.21, 0.21, 0.21, 0.21]
2637
2
3
[33.29, 33.29, 18.9, 18.9, 5.38, 5.38, 5.38, 5.38]
2815
2
3
[21.43, 21.43, 21.43, 21.43, 21.43, 21.43, 21.43, 21.43]
3136
2
3
[21.87, 18.53, 18.53, 18.53]
3221
2
3
[29.36, 15.92, 15.92, 15.92]
3504
2
3
[91.46, 91.46, 25.37, 25.37, 13.13, 13.13, 13.13, 13.13]
3609
2
3
[33.47, 29.7, 29.7, 29.7]
4532
2
3
[31.

In [ ]:
df.head()

,Unnamed: 0.1,Unnamed: 0,node_code,node_type,wlan_code,x(m),y(m),z(m),central_freq(GHz),channel_bonding_model,...,airtime,int_map,sinr_mean,sinr_std,rssi_mean,rssi_std,distance,distance_mean,distance_std,num_stations
0,0,0,AP_A,0,A,7.5000,8.3333,8.3417,5.0,4,...,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",27.009,27.375253,-64.182,64.328276,NaN,7.816307,8.088183,10.0
1,1,1,STA_A1,1,A,12.0627,4.6918,5.0670,5.0,4,...,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",NaN,NaN,NaN,NaN,6.693461,NaN,NaN,NaN
2,2,2,STA_A2,1,A,8.2712,4.8383,9.9676,5.0,4,...,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",NaN,NaN,NaN,NaN,3.931072,NaN,NaN,NaN
3,3,3,STA_A3,1,A,8.4824,3.5287,8.9153,5.0,4,...,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",NaN,NaN,NaN,NaN,4.937439,NaN,NaN,NaN
4,4,4,STA_A4,1,A,1.3639,1.8250,8.3635,5.0,4,...,"[50.3, 50.3, 50.3, 50.3]","[inf, -79.65, -93.86, -109.01, -78.68, -85.3, ...",NaN,NaN,NaN,NaN,8.944840,NaN,NaN,NaN


In [ ]:
df["airtime_0"] = None
df["airtime_1"] = None
df["airtime_2"] = None
df["airtime_3"] = None
df["airtime_4"] = None
df["airtime_5"] = None
df["airtime_6"] = None
df["airtime_7"] = None

In [ ]:
df["airtime_"+str(0)][0]

In [ ]:
eval(df["airtime"][0])[0]

50.3

In [ ]:
for idx,val in enumerate(df["airtime"]):
    for i in range(len(eval(val))):
        df.at[idx,"airtime_"+str(i)] = eval(val)[i]

In [ ]:
df.head(30)

,Unnamed: 0.1,Unnamed: 0,node_code,node_type,wlan_code,x(m),y(m),z(m),central_freq(GHz),channel_bonding_model,...,distance_std,num_stations,airtime_0,airtime_1,airtime_2,airtime_3,airtime_4,airtime_5,airtime_6,airtime_7
0,0,0,AP_A,0,A,7.5000,8.3333,8.3417,5.0,4,...,8.088183,10.0,50.3,50.3,50.3,50.3,None,None,None,None
1,1,1,STA_A1,1,A,12.0627,4.6918,5.0670,5.0,4,...,NaN,NaN,50.3,50.3,50.3,50.3,None,None,None,None
2,2,2,STA_A2,1,A,8.2712,4.8383,9.9676,5.0,4,...,NaN,NaN,50.3,50.3,50.3,50.3,None,None,None,None
3,3,3,STA_A3,1,A,8.4824,3.5287,8.9153,5.0,4,...,NaN,NaN,50.3,50.3,50.3,50.3,None,None,None,None
4,4,4,STA_A4,1,A,1.3639,1.8250,8.3635,5.0,4,...,NaN,NaN,50.3,50.3,50.3,50.3,None,None,None,None
5,5,5,STA_A5,1,A,1.3786,3.2654,8.2911,5.0,4,...,NaN,NaN,50.3,50.3,50.3,50.3,None,None,None,None
6,6,6,STA_A6,1,A,1.0444,1.2486,8.0232,5.0,4,...,NaN,NaN,50.3,50.3,50.3,50.3,None,None,None,None
7,7,7,STA_A7,1,A,0.0853,13.7302,8.3594,5.0,4,...,NaN,NaN,50.3,50.3,50.3,50.3,None,None,None,None
8,8,8,STA_A8,1,A,7.8169,18.2302,5.4278,5.0,4,...,NaN,NaN,50.3,50.3,50.3,50.3,None,None,None,None
9,9,9,STA_A9,1,A,9.9062,5.0489,3.0879,5.0,4,...,NaN,NaN,50.3,50.3,50.3,50.3,None,None,None,None


In [ ]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'node_code', 'node_type', 'wlan_code',
       'x(m)', 'y(m)', 'z(m)', 'central_freq(GHz)', 'channel_bonding_model',
       'primary_channel', 'min_channel_allowed', 'max_channel_allowed',
       'tpc_default(dBm)', 'cca_default(dBm)', 'traffic_model',
       'traffic_load[pkt/s]', 'packet_length', 'num_packets_aggregated',
       'capture_effect_model', 'capture_effect_thr', 'constant_per',
       'pifs_activated', 'cw_adaptation', 'cont_wind', 'cont_wind_stage',
       'throughput', 'sinr', 'rssi', 'airtime', 'int_map', 'sinr_mean',
       'sinr_std', 'rssi_mean', 'rssi_std', 'distance', 'distance_mean',
       'distance_std', 'num_stations', 'airtime_0', 'airtime_1', 'airtime_2',
       'airtime_3', 'airtime_4', 'airtime_5', 'airtime_6', 'airtime_7'],
      dtype='object')

In [ ]:
ramon_df = df.copy()

In [ ]:
# ramon_df.drop(["Unnamed: 0.1"],axis=1,inplace=True)

In [ ]:
ramon_df.drop(['Unnamed: 0', 'node_code', 'wlan_code',
       'x(m)', 'y(m)', 'z(m)', 'central_freq(GHz)', 'channel_bonding_model',
       'primary_channel', 'min_channel_allowed', 'max_channel_allowed',
       'tpc_default(dBm)', 'cca_default(dBm)', 'traffic_model',
       'traffic_load[pkt/s]', 'packet_length', 'num_packets_aggregated',
       'capture_effect_model', 'capture_effect_thr', 'constant_per',
       'pifs_activated', 'cw_adaptation', 'cont_wind', 'cont_wind_stage',
       'throughput', 'sinr', 'rssi', 'airtime', 'int_map'],axis=1,inplace=True)

In [ ]:
ramon_df.drop(["distance"],axis=1,inplace=True)

In [ ]:
# ramon_df.dropna(inplace=True)

In [ ]:
ramon_df = ramon_df.drop(ramon_df[ramon_df["node_type"]==1].index)

In [ ]:
ramon_df.head()

,Unnamed: 0.1,node_type,sinr_mean,sinr_std,rssi_mean,rssi_std,distance_mean,distance_std,num_stations,airtime_0,airtime_1,airtime_2,airtime_3,airtime_4,airtime_5,airtime_6,airtime_7
0,0,0,27.009000,27.375253,-64.182000,64.328276,7.816307,8.088183,10.0,50.30,50.30,50.30,50.30,0.00,0.00,0.00,0.00
11,11,0,18.855333,19.499881,-67.495333,67.686723,8.046795,8.382189,15.0,24.50,24.50,24.50,24.50,24.50,24.50,24.50,24.50
27,27,0,23.541875,24.239794,-62.785000,62.937206,7.123930,7.417081,16.0,26.46,19.73,19.73,19.73,0.00,0.00,0.00,0.00
44,44,0,23.649167,23.949777,-64.905000,65.026222,8.161596,8.388810,12.0,63.19,61.84,49.57,49.57,32.83,32.83,32.83,32.83
57,57,0,20.869167,21.748514,-65.165833,65.297246,8.313973,8.564038,12.0,27.15,19.76,19.76,19.76,0.00,0.00,0.00,0.00


In [ ]:
ramon_df.shape[0]

19200

In [ ]:
ramon_df.drop(["Unnamed: 0.1"],axis=1,inplace=True)

In [ ]:
ramon_df.reset_index(inplace=True)
ramon_df.head()

,index,node_type,sinr_mean,sinr_std,rssi_mean,rssi_std,distance_mean,distance_std,num_stations,airtime_0,airtime_1,airtime_2,airtime_3,airtime_4,airtime_5,airtime_6,airtime_7
0,0,0,27.009000,27.375253,-64.182000,64.328276,7.816307,8.088183,10.0,50.30,50.30,50.30,50.30,0.00,0.00,0.00,0.00
1,11,0,18.855333,19.499881,-67.495333,67.686723,8.046795,8.382189,15.0,24.50,24.50,24.50,24.50,24.50,24.50,24.50,24.50
2,27,0,23.541875,24.239794,-62.785000,62.937206,7.123930,7.417081,16.0,26.46,19.73,19.73,19.73,0.00,0.00,0.00,0.00
3,44,0,23.649167,23.949777,-64.905000,65.026222,8.161596,8.388810,12.0,63.19,61.84,49.57,49.57,32.83,32.83,32.83,32.83
4,57,0,20.869167,21.748514,-65.165833,65.297246,8.313973,8.564038,12.0,27.15,19.76,19.76,19.76,0.00,0.00,0.00,0.00


In [ ]:
ramon_df.drop(["index"],axis=1,inplace=True)
ramon_df.head()

,node_type,sinr_mean,sinr_std,rssi_mean,rssi_std,distance_mean,distance_std,num_stations,airtime_0,airtime_1,airtime_2,airtime_3,airtime_4,airtime_5,airtime_6,airtime_7
0,0,27.009000,27.375253,-64.182000,64.328276,7.816307,8.088183,10.0,50.30,50.30,50.30,50.30,0.00,0.00,0.00,0.00
1,0,18.855333,19.499881,-67.495333,67.686723,8.046795,8.382189,15.0,24.50,24.50,24.50,24.50,24.50,24.50,24.50,24.50
2,0,23.541875,24.239794,-62.785000,62.937206,7.123930,7.417081,16.0,26.46,19.73,19.73,19.73,0.00,0.00,0.00,0.00
3,0,23.649167,23.949777,-64.905000,65.026222,8.161596,8.388810,12.0,63.19,61.84,49.57,49.57,32.83,32.83,32.83,32.83
4,0,20.869167,21.748514,-65.165833,65.297246,8.313973,8.564038,12.0,27.15,19.76,19.76,19.76,0.00,0.00,0.00,0.00


In [ ]:
ramon_df.isna().sum()

node_type            0
sinr_mean        13973
sinr_std         13973
rssi_mean            0
rssi_std             0
distance_mean        0
distance_std         0
num_stations         0
airtime_0            0
airtime_1            0
airtime_2            0
airtime_3            0
airtime_4            0
airtime_5            0
airtime_6            0
airtime_7            0
dtype: int64

In [ ]:
df.isna().sum()

Unnamed: 0.1                   0
Unnamed: 0                     0
node_code                      0
node_type                      0
wlan_code                      0
x(m)                           0
y(m)                           0
z(m)                           0
central_freq(GHz)              0
channel_bonding_model          0
primary_channel                0
min_channel_allowed            0
max_channel_allowed            0
tpc_default(dBm)               0
cca_default(dBm)               0
traffic_model                  0
traffic_load[pkt/s]            0
packet_length                  0
num_packets_aggregated         0
capture_effect_model           0
capture_effect_thr             0
constant_per                   0
pifs_activated                 0
cw_adaptation                  0
cont_wind                      0
cont_wind_stage                0
throughput                     0
sinr                      112961
rssi                           0
airtime                        0
int_map   

In [ ]:
df["sinr"].max()

54.49

In [ ]:
print(df["sinr"][248])
print(df["node_code"][248])
print(df["rssi"][248])

31.94
STA_E7
-60.24


In [ ]:
for idx,val in df.iterrows():
    if np.isnan(df.at[idx,"sinr"]):
        print(idx)

244
245
246
247
249
253
255
257
258
350
418
419
420
422
423
424
426
427
428
429
430
431
432
433
434
435
436
437
484
505
506
508
587
592
593
594
596
598
599
600
601
602
603
619
633
656
658
713
716
717
721
794
804
836
837
839
841
842
846
847
848
851
854
856
857
858
859
860
864
865
866
867
868
869
871
872
873
926
930
937
1045
1061
1063
1067
1078
1079
1104
1159
1181
1202
1210
1212
1216
1218
1219
1223
1247
1254
1323
1379
1380
1384
1387
1447
1511
1539
1880
1944
1952
2005
2104
2106
2110
2111
2113
2117
2118
2120
2122
2141
2188
2193
2195
2196
2197
2198
2199
2201
2202
2203
2204
2205
2206
2207
2208
2211
2213
2214
2215
2216
2225
2228
2229
2232
2233
2234
2235
2238
2243
2244
2246
2247
2250
2251
2255
2257
2259
2337
2390
2418
2527
2548
2558
2560
2563
2598
2616
2714
2716
2718
2719
2720
2723
2766
2776
2804
2848
2849
2850
2852
2853
2855
2856
2859
2892
2904
2951
2953
2959
2977
2979
3091
3101
3108
3169
3170
3171
3172
3173
3176
3178
3183
3184
3186
3254
3256
3258
3265
3266
3372
3438
3443
3466
3467
3475
3477


In [ ]:
#NaN SINR
#not matching min channel max channel
#RSSI replaced between -20 and -10
#variability
#interference/obstacles